<a href="https://colab.research.google.com/github/tumble-weed/LSTM-from-scratch/blob/master/PseudoLSTM_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import numpy as np


In [0]:
class PseudoLSTM(torch.nn.Module):
    def __init__(self,state_size,input_size,state_squash_fn = torch.nn.functional.tanh):
        torch.nn.Module.__init__(self)
        self.state_size =state_size
        self.input_size = input_size
        self.state_squash_fn = state_squash_fn
        
        #-----------------------------------------------------------------------
        # 'o' (Read) gate
        self.W_x_o = torch.nn.Linear(input_size,state_size)
        self.W_s_o = torch.nn.Linear(state_size,state_size,bias=False)

        #-----------------------------------------------------------------------
        # 'i' (Write) gate
        self.W_x_i = torch.nn.Linear(input_size,state_size)
        self.W_s_i = torch.nn.Linear(state_size,state_size,bias=False)

        #-----------------------------------------------------------------------
        # 'r' (Remember) gate
        self.W_x_r = torch.nn.Linear(input_size,state_size)
        self.W_s_r = torch.nn.Linear(state_size,state_size,bias=False)

        #-----------------------------------------------------------------------
        # candidate write
        self.W_x_s = torch.nn.Linear(input_size,state_size)
        self.W_s_s = torch.nn.Linear(state_size,state_size,bias=False)
        
        pass
    def forward(self,x,s):
        # x.shape  = n_time_steps, batch_size,input_size
        outs = []
        for x_t in x:
            # gates
            squashed_s = self.state_squash_fn(s)
            o_t = self.W_x_o(x_t) + self.W_s_o(squashed_s)
            r_t = self.W_x_r(x_t) + self.W_s_r(squashed_s)
            i_t = self.W_x_i(x_t) + self.W_s_i(squashed_s)
            
            s_hat = self.W_x_s(x_t) + self.W_s_s(o_t*squashed_s)
            s = s * r_t + s_hat * i_t
            outs.append(s)
        out = torch.stack(outs,0)
        return out,s
        

In [0]:
input_size,seq_len,batch_size = 10,7,32
x = torch.zeros(seq_len,batch_size,input_size,)
state_size = 50
pseudo_lstm = PseudoLSTM(state_size,input_size,state_squash_fn = torch.nn.functional.tanh)
outs, s = pseudo_lstm(x,s = torch.zeros(batch_size,state_size))

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
